#ACTIVIDAD 03
---
```
Universidad Nacional de San Antonio Abad del Cusco
Asignatura: Mineria de Datos
Docente   : Carlos Fernando Montoya Cubas
Autor     : Etson Ronaldao Rojas Cahuana
Fecha     : 24/11/2021
Lugar     : Cusco, Perú
Proposito : Implementar el algoritmo apriori
```
---

# Introducción

Esta tarea consiste en implementar el algoritmo Apriori, propuesto por Agrawal y
Srikant en 1994. Este tiene como objetivo encontrar itemsets frecuentes dentro de una base de datos y generar reglas de asociación bajo determinados umbrales de soporte y confianza.

En data mining, las reglas de asociación son ampliamente utilizadas para descubrir relaciones entre variables en bases de datos de gran tamaño. Aplicaciones clásicas de este tipo de estrategias pueden ser encontradas en análisis de compras y de características socio-demográficas desde bases de datos censales, entre otras.

# Base de Datos

La base de datos a utilizar corresponde a múltiples playlists de la plataforma spotify creadas por usuarios de esta. Este es una muestra del dataset publicado para el Rec- SysChallenge 2018.

**spotify.npy** : Cada una de sus filas contiene una lista de canciones, la que
representa una playlist de spotify.

#Implementacion

#Librerias

*   Pandas: Es una biblioteca de software escrita como extensión de NumPy para manipulación y análisis de datos. 
*   Numpy: Da soporte para crear vectores y matrices grandes multidimensionales, junto con una gran colección de funciones matemáticas de alto nivel para operar con ellas.
*   Collections - defaultdict: Collections ayudara al manejo de los diccionarios y tuplas, y defaultdict creará cualquier elemento al que se intente acceder (siempre que, por supuesto, aún no existe), para evitar el keyrror del diccionario.








In [1]:
import pandas as pd
import numpy as np
import collections
from collections import defaultdict

#Funciones 

##Principales
*   **get_frequent_itemsets(playlists, min_support):** Recibe la estructura de datos que contiene a las playlists y retorna una estructura con los itemsets fre-cuentes, bajo un umbral mínimo de soporte.

*   **generate_association_rules(frequent_itemsets, confidence, lift):** Recibe los itemsets frecuentes generados por la función anterior y retorna las reglas de asociación. Se le puede entregar umbrales de confianza o lift para las reglas que se retornarán. Por ejemplo, si se llama esta función con los ar- gumentos confidence = 0.5 y lift = 1.2, se espera que se retornen aquellas reglas que cumplan con una confianza ≥ 0.5 y un lift ≥ 1.2.


Primera funcion principal

In [12]:
def get_frequent_itemsets(playlists, min_support):
  #Se convierte los datos en listas de conjuntos, tambien se genera la frecuencia de cada cancion
  playlists = list(playlists.item().values())
  playlists = [set(playlist) for playlist in playlists]
  canciones = [item for sublist in playlists for item in sublist]
  contador_canciones = pd.Series(data=canciones).value_counts().to_dict()
  #Se genera un diccionario de canciones y los indices de las playlists en las que tambien estan incluidas
  canciones_playlists = collections.defaultdict(set)
  for index, playlist in enumerate(playlists):
      for cancion in playlist:
          canciones_playlists[cancion].add(index)
  canciones_playlists = canciones_playlists
  #Se obtienen los itemsets de las canciones que aparecen un porcentaje de veces mayor al umbral minimo de soporte
  _contador = {cancion: veces for cancion, veces in contador_canciones.items() if veces / len(playlists) >= min_support}
  _itemset = [{cancion} for cancion in _contador.keys()]
  #Se obtienen los itemsets mas frecuentes
  _itemsets_frecuentes = {}
  _itemsets_frecuentes[1] = sorted(_contador.items(), key=lambda x: x[1], reverse=True)
  _itemsets_frecuentes_size = []
  k = 2
  actual = _itemset   #current :actual
  while len(actual) != 0:
    #Generar itemsets candidatos
    combinaciones = set() 
    m = k - 2
    for candidato in actual: 
      candidato = list(candidato)
      for aux_candidato in actual:
        aux_candidato = list(aux_candidato)
        union = True
        for i in range(k - 2):
          if candidato[i] != aux_candidato[i]:
              union = False
              break
        if not union:
          continue
        if candidato[k - 2] < aux_candidato[k - 2]:
          c = candidato + [aux_candidato[k - 2]]
          c = frozenset(sorted(c))
          combinaciones.add(c)
    #Generar el itemset que cumplan con la condicion del umbral minimo de soporte
    contador_combinaciones = {}   
    playlist_tamanio = len(playlists)   
    for candidato in combinaciones:
        playlists_inter = []
        for song in candidato:
          playlists_inter.append(canciones_playlists[song])
        contador_combinaciones[candidato] = len(set.intersection(*playlists_inter))
    contador_l = {subset: veces for subset, veces in contador_combinaciones.items() if veces / playlist_tamanio >= min_support} 
    diccionario_l = contador_l.keys() 
    _itemsets_frecuentes_size.extend(diccionario_l)
    _itemsets_frecuentes[k] = sorted(contador_l.items(), key=lambda x: x[1], reverse=True)
    k += 1
    actual = diccionario_l
  #Generar los itemsets en un dataframe
  _itemsets_df = pd.DataFrame([item for sublist in _itemsets_frecuentes.values() for item in sublist]).round(3)
  _itemsets_df.columns = ["itemset", "contador_support"]
  _itemsets_df["support"] = _itemsets_df["contador_support"] / len(playlists)
  #La funcion retornara un vector con 4 datos
  extra_data = []
  #En la primera posicion estaran los itemsets mas frecuentes.
  extra_data.append(_itemsets_df)
  #En la segunda posicion estaran los datos de la playlist para un posterior uso.
  extra_data.append(playlists)
  #En la tercera posicion estara el registro de las canciones que estan en las playlists.
  extra_data.append(canciones_playlists)
  #En la cuarta posicion estaran el registro de los itemsets largos para un posterior uso.
  extra_data.append(_itemsets_frecuentes_size)
  return extra_data

Segunda funcion principal

In [4]:
#Funcion para generar las reglas de asociacion
def generate_association_rules(frequent_itemsets, confidence, lift):
  reglas = []
  for itemset in frequent_itemsets[3]:
    #Se forman todas las combinaciones posibles de X -> Y, donde X ⊂ I y Y = I - X
    itemset_contador = calcular_inter(itemset,frequent_itemsets[2])
    itemset_support = itemset_contador / len(frequent_itemsets[0])
    for i in range(1, len(itemset) + 1):
        for x_set in combinations(itemset, i):
            x_set = set(x_set)
            y_set = set(itemset) - x_set
            x_support = calcular_inter(x_set,frequent_itemsets[2]) / len(frequent_itemsets[0])
            x_y_support = calcular_inter(x_set.union(y_set),frequent_itemsets[2]) / len(frequent_itemsets[0])
            rule_confidence = x_y_support / x_support
            if len(x_set) > 0 and len(y_set) > 0:
                y_support = calcular_inter(y_set,frequent_itemsets[2]) / len(frequent_itemsets[0])
                rule_lift = x_y_support / (x_support * y_support)
                reglas.append((x_set, y_set, rule_confidence, x_y_support, rule_lift))
  #Generar dataframe de las reglas de asociacion
  reglas_df = pd.DataFrame(data=reglas, columns=["X", "Y", "confidence", "support", "lift"]).round(3)
  reglas_df["X"] = list(map(tuple, reglas_df["X"]))
  reglas_df["Y"] = list(map(tuple, reglas_df["Y"]))

  #Hallar las reglas de asociacion de 10 elementos para la confianza
  order_by="confidence"
  n=10
  confidence_df = reglas_df.sort_values(order_by, ascending=False).head(n)

  #Hallar las reglas de asociacion de 10 elementos para lift
  order_by="lift"
  n=10
  lift_df2 = reglas_df.sort_values(order_by, ascending=False).head(n)

  #Hallar las reglas de asociacion que cumplan con la condicion de confianza y lift
  reglas_df["len_itemset"] = reglas_df.apply(lambda x: len(set(x["X"]).union(set(x["Y"]))), axis=1)
  reglas_df=reglas_df.sort_values(by="len_itemset", ascending=False)
  both_rules = reglas_df[(reglas_df["confidence"] >= confidence) & (reglas_df["lift"] >= lift)]

  return confidence_df,lift_df2,both_rules

##Secundarios

*   **calcular_inter(subset,canciones_playlist):** Recibe las tuplas de los itemsets para calcular el contador de soporte de un itemset con la lista de canciones en playlist. Una vez obtenidas las listas de reproducción en que aparece cada canción del itemset, se interseca con las listas de reproduccion de playlists. Todos los conjuntos intersecados seran el soporte de determinado itemset.

*   **combinations(iterable, r):** Este modulo sirve para generar las tuplas de combinaciones de los itemsets.
Por ejemplo: 

  ```
  Combinations('ABCD', 2) --> AB AC AD BC BD CD
  Combinations(range(4), 3) --> 012 013 023 123
  ```

 Esta ultima implementación fue extraída de [itertools](https://docs.python.org/3/library/itertools.html#itertools.combinations).




Funciones secundarias

In [5]:
#Modulo para crear interseccion de los itemset
def calcular_inter(subset,canciones_playlist):
    playlists_inter = []
    for song in subset:
        playlists_inter.append(canciones_playlist[song])
    return len(set.intersection(*playlists_inter))
    
#Modulo para generar combinaciones en los itemsets    
def combinations(iterable, r):
    pool = tuple(iterable)
    n = len(pool)
    if r > n:
        return
    indices = list(range(r))
    yield tuple(pool[i] for i in indices)
    while True:
        for i in reversed(range(r)):
            if indices[i] != i + n - r:
                break
        else:
            return
        indices[i] += 1
        for j in range(i + 1, r):
            indices[j] = indices[j - 1] + 1
        yield tuple(pool[i] for i in indices)

#Aplicar algoritmo y obtener reglas de asociacion

Lectura de datos

In [6]:
db = np.load("spotify.npy", allow_pickle = True)

Generar itemsets con un umbral de soporte bajo (0.01) para eliminar reglas poco interesantes y elegir itemsets que no ocurrieron al azar.

In [21]:
itemsets = get_frequent_itemsets(db,0.01)

10 itemsets frecuentes con mayor soporte.

In [14]:
itemsets[0].sort_values(by="support").head()

,itemset,contador_support,support
1023,"(HUMBLE., DNA., Mask Off)",100,0.01
999,"(Black Beatles, No Problem (feat. Lil Wayne & ...",100,0.01
998,"(Fake Love, Black Beatles)",100,0.01
997,"(Jumpman, One Dance)",100,0.01
996,"(Drowning (feat. Kodak Black), goosebumps)",100,0.01


Generar reglas de asociacion con una confianza de 50% y 90% de lift.

In [22]:
reglas_confidence,reglas_lift,reglas_ambos = generate_association_rules(itemsets,0.5,0.9)

 10 reglas ordenadas con mayor confidence

In [23]:
reglas_confidence

,X,Y,confidence,support,lift
441,"(DNA., Mask Off)","(HUMBLE.,)",0.909,0.098,2.002
375,"(DNA., XO TOUR Llif3)","(HUMBLE.,)",0.864,0.100,1.904
45,"(DNA.,)","(HUMBLE.,)",0.823,0.186,1.811
357,"(XO TOUR Llif3, Mask Off)","(HUMBLE.,)",0.804,0.128,1.770
351,"(Bounce Back, Broccoli (feat. Lil Yachty))","(Bad and Boujee (feat. Lil Uzi Vert),)",0.775,0.098,2.301
423,"(Slippery (feat. Gucci Mane), XO TOUR Llif3)","(HUMBLE.,)",0.765,0.099,1.685
381,"(XO TOUR Llif3, Tunnel Vision)","(HUMBLE.,)",0.750,0.100,1.652
363,"(Congratulations, Mask Off)","(HUMBLE.,)",0.747,0.118,1.645
417,"(goosebumps, Mask Off)","(HUMBLE.,)",0.743,0.107,1.637
399,"(Bounce Back, Mask Off)","(HUMBLE.,)",0.743,0.099,1.635


 10 reglas ordenadas con mayor lift

In [24]:
reglas_lift

,X,Y,confidence,support,lift
123,"(No Heart,)","(X (feat. Future),)",0.696,0.101,3.493
122,"(X (feat. Future),)","(No Heart,)",0.505,0.101,3.493
226,"(Chicken Fried,)","(Knee Deep (feat. Jimmy Buffett),)",0.500,0.104,3.261
227,"(Knee Deep (feat. Jimmy Buffett),)","(Chicken Fried,)",0.682,0.104,3.261
20,"(You Was Right,)","(Money Longer,)",0.603,0.114,3.183
21,"(Money Longer,)","(You Was Right,)",0.603,0.114,3.183
280,"(Bank Account,)","(Butterfly Effect,)",0.483,0.108,2.547
281,"(Butterfly Effect,)","(Bank Account,)",0.572,0.108,2.547
276,"(Bank Account,)","(Magnolia,)",0.491,0.110,2.541
277,"(Magnolia,)","(Bank Account,)",0.571,0.110,2.541


 10 reglas que cumplen con el limite mayor a los umbrales de confidence y lift al mismo tiempo.

In [25]:
reglas_ambos.head(10)

,X,Y,confidence,support,lift,len_itemset
447,"(XO TOUR Llif3, Congratulations)","(goosebumps,)",0.581,0.102,1.932,3
362,"(HUMBLE., Mask Off)","(Congratulations,)",0.593,0.118,1.507,3
367,"(XO TOUR Llif3, Congratulations)","(Mask Off,)",0.598,0.104,1.937,3
368,"(XO TOUR Llif3, Mask Off)","(Congratulations,)",0.656,0.104,1.668,3
369,"(Congratulations, Mask Off)","(XO TOUR Llif3,)",0.660,0.104,2.094,3
373,"(HUMBLE., DNA.)","(XO TOUR Llif3,)",0.537,0.100,1.702,3
374,"(HUMBLE., XO TOUR Llif3)","(DNA.,)",0.500,0.100,2.216,3
375,"(DNA., XO TOUR Llif3)","(HUMBLE.,)",0.864,0.100,1.904,3
379,"(HUMBLE., XO TOUR Llif3)","(Tunnel Vision,)",0.500,0.100,2.073,3
380,"(HUMBLE., Tunnel Vision)","(XO TOUR Llif3,)",0.734,0.100,2.326,3


#Obtener 4 reglas
4 reglas que presentan el valor más alto de lift.

In [32]:
top_4=reglas_lift.head(4)
top_4

,X,Y,confidence,support,lift
123,"(No Heart,)","(X (feat. Future),)",0.696,0.101,3.493
122,"(X (feat. Future),)","(No Heart,)",0.505,0.101,3.493
226,"(Chicken Fried,)","(Knee Deep (feat. Jimmy Buffett),)",0.500,0.104,3.261
227,"(Knee Deep (feat. Jimmy Buffett),)","(Chicken Fried,)",0.682,0.104,3.261


#Explicar reglas obtenidas

Una vez obtenidas las 4 reglas a analizar, tomamos en cuenta que se hizo una filtracion del valor lift mas alto ya que nos va arrojar itemsets que seran mas claros y da con mayor certeza la aparicion frecuente de la tupla de las canciones en playlists y que estas no sean simplemente coincidencias.

Para considerar sobre las reglas obtenidas:
1.   Se puso un valor de umbral de soporte bajo (0.01) para que se pueda generar itemsets interesantes.
2.   El valor de lift muy alejado de 1 indica que esta regla no se genero al azar o por simples coincidencias.





**Primera regla**

```
(No Heart,) ---> (X (feat. Future),)
```
Se toma en cuenta que ambas canciones son de genero rap y su relacion puede estar ligada al gusto musical de este genero.
El valor de la confianza de la regla alcanza 0.696 y supera ligeramente el umbral definido (0.5), se puede decir que la regla se cumple según ese criterio. 



**Segunda regla**
```
(X (feat. Future),) ---> (No Heart,)	
```
Se vuelve a repetir las mismas canciones de la anterior regla, podemos plantear que estas canciones estan ligadas a los gustos musicales de los amantes del rap o hip-hop.
De la misma manera en la que se analizo la anterior cancion, la confianza en esta regla supera el umbral predefinido y cumple con el criterio.

**Tercera regla**

```
(Chicken Fried,) ---> (Knee Deep (feat. Jimmy Buffett),)	
```
Estas canciones son de genero country - folk, segun lo revisado, ambas canciones pertenecen a la misma banda, por lo cual su relacion esta ligada al gusto musical de la banda, o tambien puede resultar que ambas canciones son las mas exitosas de este grupo y tienen mayor aprobacion.
El umbral de confianza de aparicion de esta regla se iguala con la definida y la cumple. 

**Cuarta regla**

```
(Knee Deep (feat. Jimmy Buffett),) ---> (Chicken Fried,)	
```
Esta cancion vuelve a aparecer al igual que la anterior regla por lo que se tiene una fuerte relacion y podemos concluir que no es una simple coincidencia. Pero en este caso la confianza alcanza 0.682 y supera el umbral definido, concluyendo que la regla ofrece mayor fidelidad gracias a este criterio.

#Analisis de frecuencia de canciones

Mostramos las 20 canciones mas frecuentes en todas las playlists que se registraron.

In [34]:
playlists = db.item().values()
_canciones = [item for sublist in playlists for item in sublist]
pd.Series(_canciones).value_counts().sort_values(ascending=False)[0:20]

Closer                                 761
Home                                   489
HUMBLE.                                470
Roses                                  436
One Dance                              433
Ride                                   429
Congratulations                        410
Let Me Love You                        403
Broccoli (feat. Lil Yachty)            402
Caroline                               395
Gold                                   386
T-Shirt                                355
Stay                                   355
Bad and Boujee (feat. Lil Uzi Vert)    353
Riptide                                339
iSpy (feat. Lil Yachty)                339
Forever                                337
Unforgettable                          331
Sorry                                  330
XO TOUR Llif3                          330
dtype: int64

Aunque la cancion Closer del grupo Chainsmokers aparezca con mayor frecuencia en las playlists, no significa que esta, este relacionada de manera especial o unica con otra cancion, ya que no se observo alguna relacion en las reglas, por lo que se concluye que es un single que tuvo exito y no hay dependencias.